In [1]:
import os
import pandas as pd
from carn_utils.carn_dataloader import copy_by_fold
from carn_utils.split_utils import carn_split_keys, export_split_tsv


In [2]:
# Run the function with your parameters
dataset_path = '../../datasets'
output_dir = 'data'

## Carnatic Dataloader

In [3]:
#copy_by_fold(dataset_path=dataset_path, validate=False, verbose=True)

seed = 0
folds = [1,2]

for fold in folds:
    print(f"Processing fold {fold}...")
    # Get the split keys for the current fold
    export_split_tsv(dataset_path=dataset_path, train_fold=fold, seed=seed, csv_path='cmr_splits.csv')

## Preprocess

!python launch_scripts/preprocess_audio.py --orig_audio_paths data/audio_paths.tsv

## Finetune the model

In [10]:
!python launch_scripts/train.py --name carnatic-finetune --checkpoint final0 --annotation-dir cmr-fold1 --gpu 0 --use-cpu

Disabling MPS fallback for PyTorch.
Seed set to 0
Starting a new run with the following parameters:
Namespace(name='carnatic-finetune', gpu=0, force_flash_attention=False, compile=['frontend', 'transformer_blocks', 'task_heads'], n_layers=6, transformer_dim=512, frontend_dropout=0.1, transformer_dropout=0.2, lr=0.0008, weight_decay=0.01, logger='none', num_workers=8, n_heads=16, fps=50, loss='shift_tolerant_weighted_bce', warmup_steps=1000, max_epochs=100, batch_size=8, accumulate_grad_batches=8, train_length=1500, dbn=False, eval_trim_beats=5, val_frequency=5, tempo_augmentation=True, pitch_augmentation=True, mask_augmentation=True, sum_head=True, partial_transformers=True, length_based_oversampling_factor=0.65, val=True, hung_data=False, fold=None, seed=0, checkpoint='final0', annotation_dir='cmr-fold1', use_cpu=True)
Using data directory: /Users/satyajeetprabhu/Desktop/Projects/Meter Tracking/navid/beat_this/data
Using checkpoint directory: /Users/satyajeetprabhu/Desktop/Projects/Me

In [ ]:
#!python launch_scripts/train.py --name test --checkpoint final0.ckpt --annotation-dir cmr-fold1
#!python launch_scripts/train.py --load-from-checkpoint final0 --name final0_ft --max-epochs 50 --use-cpu

Seed set to 0
Starting a new run with the following parameters:
Namespace(name='final0_ft', gpu=0, force_flash_attention=False, compile=['frontend', 'transformer_blocks', 'task_heads'], n_layers=6, transformer_dim=512, frontend_dropout=0.1, transformer_dropout=0.2, lr=0.0008, weight_decay=0.01, logger='none', num_workers=8, n_heads=16, fps=50, loss='shift_tolerant_weighted_bce', warmup_steps=1000, max_epochs=50, batch_size=8, accumulate_grad_batches=8, train_length=1500, dbn=False, eval_trim_beats=5, val_frequency=5, tempo_augmentation=True, pitch_augmentation=True, mask_augmentation=True, sum_head=True, partial_transformers=True, length_based_oversampling_factor=0.65, val=True, hung_data=False, fold=None, seed=0, load_from_checkpoint='final0', use_cpu=True)
Using data directory: /Users/satyajeetprabhu/Desktop/Projects/Meter Tracking/beat_this_carnatic/data
Using checkpoint directory: /Users/satyajeetprabhu/Desktop/Projects/Meter Tracking/beat_this_carnatic/checkpoints
Validation set: 

train_fold = "cmr-fold1"  # <-- change this as needed

# Dictionary mapping fold names to their audio folder paths
available_folds = {
    "cmr-fold1": "data/audio/cmr-fold1",
    "cmr-fold2": "data/audio/cmr-fold2",
    # Add more folds here as needed
}
# Build the TSV entry
fold_audio_paths = [[train_fold, available_folds[train_fold]]]
audio_paths_df = pd.DataFrame(fold_audio_paths, columns=["fold_name", "audio_path"])
tsv_path = os.path.join(output_dir, "audio_paths.tsv")
# Write TSV with tab separator, no header or index
audio_paths_df.to_csv(tsv_path, sep="\t", index=False, header=False)

print(f"✅ Created TSV file for '{train_fold}' at: {tsv_path}")